In [2]:
from GvaDataset import GvaDataset
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms 

transform = transforms.Compose([
    transforms.Resize((1024, 512))
])

images_path = "data/_Original"
labels_path = "data/_Masks"
dataset = GvaDataset(images_path, labels_path, transform=transform)
train_loader = DataLoader(dataset, batch_size=2, shuffle=False)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/_Original'

In [1]:
img, lab, file = dataset[10]
print(img.shape)
print(lab.shape)
print(file)

NameError: name 'dataset' is not defined

In [ ]:
from models.unet import UNet
from experiment import Experiment
import torch.optim as optim
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNet(in_channels=3, out_channels=1).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()
experiment = Experiment(model, criterion, optimizer, device)

In [ ]:
experiment.train(train_loader=train_loader, epochs=10)